# Imports

In [14]:
# Dependencies and Setup

# visualizations
import matplotlib.pyplot as plt
import seaborn as sns

#data science - standard library
import pandas as pd
import numpy as np

#requests
import requests
from pprint import pprint
import time

#regression
from scipy.stats import linregress
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "booth_cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Generate City List

In [20]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

580

In [21]:
cities_sub = cities[0:10]
cities_sub

['qaanaaq',
 'ziro',
 'ushuaia',
 'mataura',
 'tuatapere',
 'punta arenas',
 'longyearbyen',
 'klaksvik',
 'hithadhoo',
 'ketchikan']

# Make Requests

In [23]:
#init lists to hold parsed data
lats = []
longs = []
temps = []
feel_likes = []
humids = []
pressures = []
speeds = []
cloudiness = []
cities_found = []

for i in range(len(cities_sub)):
    #get the city
    city = cities_sub[i]
    
    #create the url
    units = "imperial"
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&units={units}&appid={weather_api_key}"
#     print(url)

    #make the request
    response = requests.get(url)
#     print(response.status_code)
    
    #error check
    if response.status_code == 200:
        data = response.json()
#         pprint(data)
        
        try:
            #extract the data
            lat = data["coord"]["lat"]
            long = data["coord"]["lon"]
            temp = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            pressure = data["main"]["pressure"]
            speed = data["wind"]["speed"]
            clouds = data["clouds"]["all"]
            city_name = data["name"]

            #save the data
            lats.append(lat)
            longs.append(long)
            temps.append(temp)
            feel_likes.append(feels_like)
            humids.append(humidity)
            pressures.append(pressure)
            speeds.append(speed)
            cloudiness.append(clouds)
            cities_found.append(city_name)
        except Exception as e:
            print(f"Through exception for city {city}: {e}")
    
    elif response.status_code == 404:
        print(f"Missing data in OpenWeatherAPI for {city}")
    else:
        print(response.status_code)
        print("Oh gosh darnit. The API is broken. Sad Face.")
        
    #print every 5
    if (i % 5 == 0):
        print(f"Got data for city index: {i} of {len(cities_sub)}")
    
    #sleep for requests
    time.sleep(1)
        
# make the dataframe
df = pd.DataFrame()
df["City"] = cities_found
df["Latitude"] = lats
df["Longitude"] = longs
df["Temperature"] = temps
df["Feels Like"] = feel_likes
df["Humidity"] = humids
df["Pressure"] = pressures
df["Wind Speed"] = speeds
df["Cloudiness"] = speeds

df.head(10)

Got data for city index: 0 of 10
Got data for city index: 5 of 10


,City,Latitude,Longitude,Temperature,Feels Like,Humidity,Pressure,Wind Speed,Cloudiness
0,Qaanaaq,77.48,-69.36,8.47,0.09,78,1008,3.96,3.96
1,Ziro,27.63,93.83,50.65,50.41,100,1016,0.87,0.87
2,Ushuaia,-54.80,-68.30,50.00,41.52,81,1013,12.75,12.75
3,Mataura,-46.19,168.86,61.00,59.05,64,1023,3.00,3.00
4,Tuatapere,-46.13,167.68,61.00,59.05,64,1023,3.00,3.00
5,Punta Arenas,-53.15,-70.92,51.40,44.13,50,1012,6.93,6.93
6,Longyearbyen,78.22,15.64,32.00,-1.59,96,995,53.02,53.02
7,Klaksvík,62.23,-6.59,44.60,33.76,81,988,14.99,14.99
8,Hithadhoo,-0.60,73.08,83.07,87.76,72,1011,8.12,8.12
9,Ketchikan,55.34,-131.65,50.72,44.04,93,1000,11.41,11.41


In [24]:
df.to_csv(output_data_file)